# **Download dataset and unzip it**
----> **Run this code just one time!** <----

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!wget https://datasets.simula.no/downloads/kvasir-seg.zip
!unzip kvasir-seg.zip
!mv Kvasir-SEG /content/drive/MyDrive/

In [ ]:
!wget https://github.com/nikhilroxtomar/Polyp-Segmentation-using-UNET-in-TensorFlow-2.0/blob/master/files/model.h5
!wget https://github.com/nikhilroxtomar/Polyp-Segmentation-using-UNET-in-TensorFlow-2.0/blob/master/files/data.csv
!mkdir /content/drive/MyDrive/Kvasir-SEG/pretrained
!mv model.h5 /content/drive/MyDrive/Kvasir-SEG/pretrained/
!mv data.csv /content/drive/MyDrive/Kvasir-SEG/pretrained/

# **Parameters**

In [5]:
prefix_path = "/content/drive/MyDrive/Kvasir-SEG/"
images_path = prefix_path + "images"
masks_path = prefix_path + "masks"

# **Importing Necessary Libraries and Dependencies**


In [6]:
import os, re, time, json
import cv2
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
from tensorflow.keras.utils import CustomObjectScope
from tqdm import tqdm


device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# **Dataset Preparation**

In [7]:
def load_data(path, split=0.1):
    images = sorted(glob(os.path.join(path, "images/*")))
    masks = sorted(glob(os.path.join(path, "masks/*")))

    total_size = len(images)
    valid_size = int(split * total_size)
    test_size = int(split * total_size)

    train_x, valid_x = train_test_split(images, test_size=valid_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=valid_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=test_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    x = np.expand_dims(x, axis=-1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        x = tf.cast(x, tf.float32)
        y = tf.cast(y, tf.float32)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([256, 256, 3])
    y.set_shape([256, 256, 1])
    return x, y

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()

    return dataset

# **Model**

In [8]:
def conv_block(x, num_filters):
    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def build_model():
    size = 256
    num_filters = [16, 32, 48, 64]
    inputs = Input((size, size, 3))

    skip_x = []
    x = inputs
    ## Encoder
    for f in num_filters:
        x = conv_block(x, f)
        skip_x.append(x)
        x = MaxPool2D((2, 2))(x)

    ## Bridge
    x = conv_block(x, num_filters[-1])

    num_filters.reverse()
    skip_x.reverse()
    ## Decoder
    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2))(x)
        xs = skip_x[i]
        x = Concatenate()([x, xs])
        x = conv_block(x, f)

    ## Output
    x = Conv2D(1, (1, 1), padding="same")(x)
    x = Activation("sigmoid")(x)

    return Model(inputs, x)

# with tf.device('/device:GPU:0'):
if __name__ == "__main__":
  model = build_model()
  model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 256, 256, 16)         448       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 256, 256, 16)         64        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 256, 256, 16)         0         ['batch_normalization[0][0

# **Training**

In [10]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = tf.cast(x, tf.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

with tf.device('/device:GPU:0'):
  if __name__ == "__main__":
      ## Dataset
      path = "/content/drive/MyDrive/Kvasir-SEG/"
      (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(path)

      ## Hyperparameters
      batch = 8
      lr = 1e-4
      epochs = 20

      train_dataset = tf_dataset(train_x, train_y, batch=batch)
      valid_dataset = tf_dataset(valid_x, valid_y, batch=batch)

      model = build_model()

      opt = tf.keras.optimizers.Adam(lr)
      metrics = ["acc", tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), iou]
      model.compile(loss="binary_crossentropy", optimizer=opt, metrics=metrics)

      callbacks = [
          ModelCheckpoint(prefix_path + "pretrained/trained_model.h5"),
          ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
          CSVLogger(prefix_path + "pretrained/data.csv"),
          TensorBoard(),
          EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False)
      ]

      train_steps = len(train_x)//batch
      valid_steps = len(valid_x)//batch

      if len(train_x) % batch != 0:
          train_steps += 1
      if len(valid_x) % batch != 0:
          valid_steps += 1

      model.fit(train_dataset,
          validation_data=valid_dataset,
          epochs=epochs,
          steps_per_epoch=train_steps,
          validation_steps=valid_steps,
          callbacks=callbacks)

Epoch 1/20
100/100 [==============================] - ETA: 0s - loss: 0.5226 - acc: 0.7761 - recall: 0.5337 - precision: 0.3649 - iou: 0.1758

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


100/100 [==============================] - 346s 3s/step - loss: 0.5226 - acc: 0.7761 - recall: 0.5337 - precision: 0.3649 - iou: 0.1758 - val_loss: 0.5223 - val_acc: 0.8306 - val_recall: 2.4326e-05 - val_precision: 0.0682 - val_iou: 0.1292 - lr: 1.0000e-04
Epoch 2/20
100/100 [==============================] - 63s 637ms/step - loss: 0.4084 - acc: 0.8588 - recall: 0.4837 - precision: 0.5743 - iou: 0.2089 - val_loss: 0.4603 - val_acc: 0.8306 - val_recall: 1.2613e-05 - val_precision: 0.1359 - val_iou: 0.1254 - lr: 1.0000e-04
Epoch 3/20
100/100 [==============================] - 25s 252ms/step - loss: 0.3586 - acc: 0.8793 - recall: 0.4938 - precision: 0.6733 - iou: 0.2349 - val_loss: 0.4240 - val_acc: 0.8324 - val_recall: 0.0163 - val_precision: 0.7466 - val_iou: 0.1241 - lr: 1.0000e-04
Epoch 4/20
100/100 [==============================] - 26s 266ms/step - loss: 0.3278 - acc: 0.8909 - recall: 0.5224 - precision: 0.7301 - iou: 0.2580 - val_loss: 0.3841 - val_acc: 0.8511 - val_recall: 0.2392 

# **Prediction**


In [17]:
def read_image(path):
  if (type(path) != str):
    path = path.decode()
  x = cv2.imread(path, cv2.IMREAD_COLOR)
  x = cv2.resize(x, (256, 256))
  x = x/255.0
  return x

def read_mask(path):
  if (type(path) != str):
    path = path.decode()
  x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
  x = cv2.resize(x, (256, 256))
  x = np.expand_dims(x, axis=-1)
  return x

def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

with tf.device('/device:GPU:0'):
  if __name__ == "__main__":
      ## Dataset
      path = prefix_path
      batch_size = 8
      (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(path)

      test_dataset = tf_dataset(test_x, test_y, batch=batch_size)

      test_steps = (len(test_x)//batch_size)
      if len(test_x) % batch_size != 0:
          test_steps += 1

      with CustomObjectScope({'iou': iou}):
          model = tf.keras.models.load_model(prefix_path + "pretrained/trained_model.h5")
          model.evaluate(test_dataset, steps=test_steps)

      for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
          x = read_image(x)
          y = read_mask(y)
          y_pred = model.predict(np.expand_dims(x, axis=0))[0] > 0.5
          h, w, _ = x.shape
          white_line = np.ones((h, 10, 3)) * 255.0

          all_images = [
              x * 255.0, white_line,
              mask_parse(y), white_line,
              mask_parse(y_pred) * 255.0
          ]
          image = np.concatenate(all_images, axis=1)
          cv2.imwrite(f"/content/drive/MyDrive/AI_Image_Segmentaion/results/{i}.png", image)

13/13 [==============================] - 8s 568ms/step - loss: -34.8289 - acc: 0.7946 - recall: 0.5486 - precision: 0.7250 - iou: 1.5545


  0%|          | 0/100 [00:00<?, ?it/s]

1/1 [==============================] - 1s 1s/step


  1%|          | 1/100 [00:01<01:58,  1.20s/it]

1/1 [==============================] - 0s 19ms/step


...

 97%|█████████▋| 97/100 [00:11<00:00, 10.74it/s]

1/1 [==============================] - 0s 19ms/step


 99%|█████████▉| 99/100 [00:11<00:00, 10.99it/s]

1/1 [==============================] - 0s 20ms/step


100%|██████████| 100/100 [00:11<00:00,  8.47it/s]
